In [3]:
import numpy as np
import pandas as pd
import xarray

In [4]:
# glct_trans_1st = xarray.open_dataset("D:/kerja/asisten riset/code v/step 3/GbGlct/GLCT_trans_to.nc", engine="netcdf4")
gfrac = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GFRAC_NEW.NC", engine="netcdf4")
gfrac_ngfbfc = [element.strip() for element in gfrac.coords['NGFBFC'].data.astype('str').tolist()]

In [5]:
emission_CO2_netcdf = xarray.open_dataset("D:/emission_CO2_AM1_3tahun_v2.NC", engine="netcdf4")
garea = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis1_slidingwindows/GAREA.NC", engine="netcdf4")

In [6]:
def divide(arr1, arr2):
    return np.divide(arr1, arr2)

def multiple(arr1, arr2):
    return np.multiply(arr1, arr2)

In [7]:
garea_ha = np.nan_to_num(np.multiply(garea['GAREA'].isel(time=slice(0, 5)), 100))
gfrac_area = np.zeros((5, 17, 2160, 4320), dtype="float32")

for n in range(5):
    for i in range(17):
        gfrac_area[n][i] = np.nan_to_num(np.multiply(gfrac['GFRAC_new'].isel(time=n, NGFBFC=i).to_numpy(), garea_ha[n]))

In [6]:
gfrac_area_netcdf = xarray.Dataset({
    "GFRAC_combined":(["time","NGFBFC","latitude", "longitude"], gfrac_area)
},coords={
        "time": pd.date_range(start='1970-01-01', end='1990-01-01', freq='5YS'),
        "NGFBFC" : gfrac_ngfbfc,
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy()
    })

In [8]:
selisih_gfrac = np.zeros((4, 17, 2160, 4320), dtype="float32")

In [152]:
# max_year = 3

# for n in range(max_year):
#     for i, crop in enumerate(gfrac_ngfbfc):
#         if n < max_year - 1:
#             selisih_gfrac[n+1][i] = np.where((gfrac_area_netcdf["GFRAC_combined"][n+1][i] > gfrac_area_netcdf["GFRAC_combined"][n][i]),
#                                             gfrac_area_netcdf["GFRAC_combined"][n+1][i] - gfrac_area_netcdf["GFRAC_combined"][n][i], 0)

In [9]:
# =IF(AE3-AD3>0;AE3-AD3;0)
max_year = 3

for n in range(max_year):
    for i, crop in enumerate(gfrac_ngfbfc):
        selisih_gfrac[n+1][i] = np.where(gfrac_area[n+1][i] > gfrac_area[n][i], gfrac_area[n+1][i] - gfrac_area[n][i], 0)

In [12]:
selisih_gfrac[1].sum()

227397710.0

In [ ]:
# max_year = 2
# for i, crop in enumerate(gfrac_ngfbfc):
#     for n in range(max_year):
#         # if n < max_year - 1:
#         selisih_gfrac[i][n] = np.where(selisih_gfrac[i][n+1] >= selisih_gfrac[i][n], selisih_gfrac[i][n+1] - selisih_gfrac[i][n], 0)

In [13]:
selisih_gfrac_area_netcdf = xarray.Dataset({
        "selisih_gfrac":(["time", "NGFBFC", "latitude", "longitude"], selisih_gfrac)
    },
    coords={
        "NGFBFC": gfrac_ngfbfc,
        "time": pd.date_range(start='1970-01-01', end='1985-01-01', freq='5YS'),
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy(),
    })

In [14]:
proporsi_delta = np.zeros((3, 17, 2160, 4320), dtype="float32")

In [16]:
for n in range(1, 4):
    for i, crop in enumerate(gfrac_ngfbfc):
        proporsi_delta[n-1][i] = np.nan_to_num(divide(selisih_gfrac_area_netcdf['selisih_gfrac'].isel(time=n, NGFBFC=i).to_numpy(), selisih_gfrac_area_netcdf['selisih_gfrac'].isel(time=n).sum(dim='NGFBFC').to_numpy()))

C:\Users\HEFRYA~1\AppData\Local\Temp/ipykernel_12024/3062683060.py:2: RuntimeWarning: invalid value encountered in divide
  return np.divide(arr1, arr2)
C:\Users\HEFRYA~1\AppData\Local\Temp/ipykernel_12024/3062683060.py:2: RuntimeWarning: invalid value encountered in divide
  return np.divide(arr1, arr2)
C:\Users\HEFRYA~1\AppData\Local\Temp/ipykernel_12024/3062683060.py:2: RuntimeWarning: invalid value encountered in divide
  return np.divide(arr1, arr2)
C:\Users\HEFRYA~1\AppData\Local\Temp/ipykernel_12024/3062683060.py:2: RuntimeWarning: invalid value encountered in divide
  return np.divide(arr1, arr2)
C:\Users\HEFRYA~1\AppData\Local\Temp/ipykernel_12024/3062683060.py:2: RuntimeWarning: invalid value encountered in divide
  return np.divide(arr1, arr2)
C:\Users\HEFRYA~1\AppData\Local\Temp/ipykernel_12024/3062683060.py:2: RuntimeWarning: invalid value encountered in divide
  return np.divide(arr1, arr2)
C:\Users\HEFRYA~1\AppData\Local\Temp/ipykernel_12024/3062683060.py:2: RuntimeWarni

In [ ]:
# proporsi_delta_netcdf = xarray.Dataset(
# coords={
#         "time": pd.date_range(start='1975-01-01', end='1985-01-01', freq='5YS'),
#         "latitude": gfrac.coords["latitude"].to_numpy(),
#         "longitude": gfrac.coords["longitude"].to_numpy()
#     })

# coords = ("time", "latitude", "longitude")
# data_vars = {
#     ngfbc: (coords, proporsi_delta[i]) for i, ngfbc in enumerate(gfrac_ngfbfc)
# }
# proporsi_delta_netcdf = proporsi_delta_netcdf.assign(data_vars)

In [17]:
agri_to_agri_prop_crop = np.zeros((17, 3, 2160, 4320), dtype="float32")

In [18]:
for i, crop in enumerate(gfrac_ngfbfc):
    for n in range(3):
        agri_to_agri_prop_crop[i][n] = np.nan_to_num(multiple(emission_CO2_netcdf['Agricultural Transition Emission'].isel(time=n).to_numpy(), proporsi_delta[n][i]))

In [19]:
luc_agri_prop_crop_netcdf = xarray.Dataset(
coords={
        "time": pd.date_range(start='1975-01-01', end='1985-01-01', freq='5YS'),
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy(),
    })

coords = ("time", "latitude", "longitude")
data_vars = {
    ngfbc: (coords, agri_to_agri_prop_crop[i]) for i, ngfbc in enumerate(gfrac_ngfbfc)
}
luc_agri_prop_crop_netcdf = luc_agri_prop_crop_netcdf.assign(data_vars)

In [ ]:
x = [-51.706, -55.974643, 101.457, 109.38, -96.6289, 48.966]
y = [-22.698, -21.208333, 31.4420, -7.11, 18.549639, -18.07]

In [ ]:
# luc_agri_prop_crop_netcdf.isel(time=0).where(luc_agri_prop_crop_netcdf.isel(time=0)['country'] == "Brazil").sum().to_pandas().sum()

In [27]:
# x = np.random.randint(1, 2160)
# y = np.random.randint(1, 4320)
# luc_agri_prop_crop_netcdf.isel(time=2, latitude=x, longitude=y).to_pandas().sum(), emission_CO2_netcdf['Agricultural Transition Emission'].isel(time=2, latitude=x, longitude=y).sum().to_pandas().sum()

In [28]:
luc_agri_prop_crop_netcdf.isel(time=2).sum().to_pandas().sum()

0.6162067553959787

In [169]:
emission_CO2_netcdf['Agricultural Transition Emission'].isel(time=1).sum().to_pandas()

array(1.005606, dtype=float32)

In [ ]:
emission_CO2_netcdf['Agricultural Transition Emission'].isel(time=0).where(emission_CO2_netcdf['Agricultural Transition Emission'].isel(time=0)['country_name'] == "Brazil").sum()